# Part 3: Evaluate Geometrical Brownian Motion Parameters

**Notebook is under cleaning**

Import libraries

In [ ]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from src.gbm import (drift_sigma_maxlikelihood, compute_gbm_params, fit_drift_skewnormal,
                     plot_drift_distribution_fits, plot_sigma_distribution_fits,
                     linear_regression_fit, plot_kde, plot_drift_vs_sigma_fit)

# PATH TO DATA
FILEPATH = "data/all_indexes_2006-01-01_2021-12-31.pickle"

with open(FILEPATH, "rb") as f:
    data = pickle.load(f)

START_YEAR = 2006
END_YEAR = 2021

DIR_KDE_FIT = 'results/gbm_parameters/kde_fit'

Define groups of indexes:

In [ ]:
# Unites States indexes
indexes_usa = ['SPX','CCMP','RIY','RTY','RAY','RLV','RLG','NBI']

# S&P500 sectors
indexes_sp500 = ['S5COND','S5CONS','S5ENRS','S5FINL','S5HLTH','S5INFT','S5MATR','S5TELS','S5UTIL','S5INDU']

# European indexes
indexes_eu = ['DAX','CAC','UKX','BEL20','IBEX','KFX','OMX','SMI']

# Asia Pacific indexes
indexes_apac = ['AS51'] 

# Japan indexes
indexes_jp = ['NKY','TPX']

# BRIC indexes
indexes_bric = ['IBOV','NIFTY','MXIN','SHCOMP','SHSZ300'] 

# All indexes: merge all lists
indexes_all = indexes_usa + indexes_sp500 + indexes_eu + indexes_apac + indexes_jp + indexes_bric

index_list = ['CAC']

Compute GBM parameters and fit them with linear function.

In [ ]:
results_fit = []

data_index = {}

for index_name in index_list:
    
    print(f"Processing {index_name} index")
    
    df = compute_gbm_params(data, index_name)

    #data_index[index_name] = {'exp_mean_to_median': np.mean(df['rho'])/np.median(df['rho']),
    #                          'exp_mean_rho': np.mean(df['rho']),
    #                          'exp_median_rho': np.median(df['rho'])}

    ## === Drift vs sigma (volatility) === ##
    ## The scatter plot does not show any clear pattern
    a, b, r2, corr = linear_regression_fit(df = df, index_name=index_name, fit_intercept = True, huber=True)

    ## === Drift and sigma KDE plots === #
    title = 'GBM drift distribution. Index {}'.format(index_name)
    plot_kde(df.drift, title, 'drift', index_name)

    title = 'GBM volatility distribution. Index {}'.format(index_name)
    plot_kde(df.sigma, title, 'volatility', index_name) 

    ## === Skenormal fit of the drift distribution === ##
    drift_skewnorm = fit_drift_skewnormal(df.drift)
    print(drift_skewnorm)

    plot_drift_distribution_fits(df.drift, index_name)
    plot_sigma_distribution_fits(df.sigma, index_name)

    corrcoeff = np.corrcoef(np.array(df.drift, dtype=float), np.array(df.sigma, dtype=float))[0,1]


#df_fit = pd.DataFrame.from_dict(results_fit, orient='columns')
#df_fit

Select columns:

In [ ]:
columns_paper=['index_name','drift_mean','drift_std','drift_loc','drift_scale','drift_skewp','sigma_mean','sigma_alpha','sigma_beta','a','b','R2','corr_drift_sigma']
dmu_paper=dmu[columns_paper]